In [11]:
import os  
import math  
import numpy as np  
import tensorflow as tf  
import matplotlib.pyplot as plt  
  
#============================================================================  
#-----------------生成图片路径和标签的List------------------------------------  
  
train_dir = 'C:/Users/sungroup/PycharmProjects/anti_cnn/pavement'  
  
bhg = []  
label_bhg = []  
hg = []  
label_hg = []  
  
#step1：获取'G:/git_cache/anti_cnn/pavement'下所有的图片路径名，存放到  
#对应的列表中，同时贴上标签，存放到label列表中。  
def get_files(file_dir, ratio):  
    for file in os.listdir(file_dir+'/bhg'):  
        bhg.append(file_dir +'/bhg'+'/'+ file)   
        label_bhg.append(0)  
    for file in os.listdir(file_dir+'/hg'):  
        hg.append(file_dir +'/hg'+'/'+file)  
        label_hg.append(1)  
  
#step2：对生成的图片路径和标签List做打乱处理把bhg和hg合起来组成一个list（img和lab）  
    image_list = np.hstack((bhg, hg))  
    label_list = np.hstack((label_bhg, label_hg))  
  
    #利用shuffle打乱顺序  
    temp = np.array([image_list, label_list])  
    temp = temp.transpose()  
    np.random.shuffle(temp)  
      
    #从打乱的temp中再取出list（img和lab）  
    #image_list = list(temp[:, 0])  
    #label_list = list(temp[:, 1])  
    #label_list = [int(i) for i in label_list]  
    #return image_list, label_list  
      
    #将所有的img和lab转换成list  
    all_image_list = list(temp[:, 0])  
    all_label_list = list(temp[:, 1])  
  
    #将所得List分为两部分，一部分用来训练tra，一部分用来测试val  
    #ratio是测试集的比例  
    n_sample = len(all_label_list)  
    n_val = int(math.ceil(n_sample*ratio))   #测试样本数  
    n_train = n_sample - n_val   #训练样本数  
  
    tra_images = all_image_list[0:n_train]  
    tra_labels = all_label_list[0:n_train]  
    tra_labels = [int(float(i)) for i in tra_labels]  
    val_images = all_image_list[n_train:-1]  
    val_labels = all_label_list[n_train:-1]  
    val_labels = [int(float(i)) for i in val_labels]  
  
    return tra_images, tra_labels, val_images, val_labels  
      
      
#---------------------------------------------------------------------------  
#--------------------生成Batch----------------------------------------------  
  
#step1：将上面生成的List传入get_batch() ，转换类型，产生一个输入队列queue，因为img和lab  
#是分开的，所以使用tf.train.slice_input_producer()，然后用tf.read_file()从队列中读取图像  
#   image_W, image_H, ：设置好固定的图像高度和宽度  
#   设置batch_size：每个batch要放多少张图片  
#   capacity：一个队列最大多少  
def get_batch(image, label, image_W, image_H, batch_size, capacity):  
    #转换类型  
    image = tf.cast(image, tf.string)  
    label = tf.cast(label, tf.int32)  
  
    # make an input queue  
    input_queue = tf.train.slice_input_producer([image, label])  
  
    label = input_queue[1]  
    image_contents = tf.read_file(input_queue[0]) #read img from a queue    
      
#step2：将图像解码，不同类型的图像不能混在一起，要么只用jpeg，要么只用png等。  
    image = tf.image.decode_jpeg(image_contents, channels=3)   
      
#step3：数据预处理，对图像进行旋转、缩放、裁剪、归一化等操作，让计算出的模型更健壮。  
    #image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)
    image = tf.image.resize_images(image, (image_W, image_H))
    image = tf.image.per_image_standardization(image)  
  
#step4：生成batch  
#image_batch: 4D tensor [batch_size, width, height, 3],dtype=tf.float32   
#label_batch: 1D tensor [batch_size], dtype=tf.int32  
    image_batch, label_batch = tf.train.batch([image, label],  
                                                batch_size= batch_size,  
                                                num_threads= 32,   
                                                capacity = capacity)  
    #重新排列label，行数为[batch_size]  
    label_batch = tf.reshape(label_batch, [batch_size])  
    image_batch = tf.cast(image_batch, tf.float32)  
    return image_batch, label_batch              
  
#========================================================================</span>  

In [16]:
#=========================================================================  
import tensorflow as tf  
#=========================================================================  
#网络结构定义  
    #输入参数：images，image batch、4D tensor、tf.float32、[batch_size, width, height, channels]  
    #返回参数：logits, float、 [batch_size, n_classes]  
def inference(images, batch_size, n_classes):  
#一个简单的卷积神经网络，卷积+池化层x2，全连接层x2，最后一个softmax层做分类。  
#卷积层1  
#64个3x3的卷积核（3通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()  
    with tf.variable_scope('conv1') as scope:  
          
        weights = tf.Variable(tf.truncated_normal(shape=[3,3,3,64], stddev = 1.0, dtype = tf.float32),   
                              name = 'weights', dtype = tf.float32)  
          
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [64]),  
                             name = 'biases', dtype = tf.float32)  
          
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')  
        pre_activation = tf.nn.bias_add(conv, biases)  
        conv1 = tf.nn.relu(pre_activation, name= scope.name)  
          
#池化层1  
#3x3最大池化，步长strides为2，池化后执行lrn()操作，局部响应归一化，对训练有利。  
    with tf.variable_scope('pooling1_lrn') as scope:  
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME', name='pooling1')  
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')  
  
#卷积层2  
#16个3x3的卷积核（16通道），padding=’SAME’，表示padding后卷积的图与原图尺寸一致，激活函数relu()  
    with tf.variable_scope('conv2') as scope:  
        weights = tf.Variable(tf.truncated_normal(shape=[3,3,64,16], stddev = 0.1, dtype = tf.float32),   
                              name = 'weights', dtype = tf.float32)  
          
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [16]),  
                             name = 'biases', dtype = tf.float32)  
          
        conv = tf.nn.conv2d(norm1, weights, strides = [1,1,1,1],padding='SAME')  
        pre_activation = tf.nn.bias_add(conv, biases)  
        conv2 = tf.nn.relu(pre_activation, name='conv2')  
  
#池化层2  
#3x3最大池化，步长strides为2，池化后执行lrn()操作，  
    #pool2 and norm2  
    with tf.variable_scope('pooling2_lrn') as scope:  
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0,beta=0.75,name='norm2')  
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1],padding='SAME',name='pooling2')  
  
#全连接层3  
#128个神经元，将之前pool层的输出reshape成一行，激活函数relu()  
    with tf.variable_scope('local3') as scope:  
        reshape = tf.reshape(pool2, shape=[batch_size, -1])  
        dim = reshape.get_shape()[1].value  
        weights = tf.Variable(tf.truncated_normal(shape=[dim,128], stddev = 0.005, dtype = tf.float32),  
                             name = 'weights', dtype = tf.float32)  
          
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [128]),   
                             name = 'biases', dtype=tf.float32)  
          
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)  
          
#全连接层4  
#128个神经元，激活函数relu()   
    with tf.variable_scope('local4') as scope:  
        weights = tf.Variable(tf.truncated_normal(shape=[128,128], stddev = 0.005, dtype = tf.float32),  
                              name = 'weights',dtype = tf.float32)  
          
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [128]),  
                             name = 'biases', dtype = tf.float32)  
          
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')  
  
#dropout层          
    with tf.variable_scope('dropout') as scope:  
        drop_out = tf.nn.dropout(local4, 0.8)  
              
          
#Softmax回归层  
#将前面的FC层输出，做一个线性回归，计算出每一类的得分，在这里是2类，所以这个层输出的是两个得分。  
    with tf.variable_scope('softmax_linear') as scope:  
        weights = tf.Variable(tf.truncated_normal(shape=[128, n_classes], stddev = 0.005, dtype = tf.float32),  
                              name = 'softmax_linear', dtype = tf.float32)  
          
        biases = tf.Variable(tf.constant(value = 0.1, dtype = tf.float32, shape = [n_classes]),  
                             name = 'biases', dtype = tf.float32)  
          
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')  
  
    return softmax_linear  
  
#-----------------------------------------------------------------------------  
#loss计算  
    #传入参数：logits，网络计算输出值。labels，真实值，在这里是0或者1  
    #返回参数：loss，损失值  
def losses(logits, labels):  
    with tf.variable_scope('loss') as scope:  
        cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_example')  
        loss = tf.reduce_mean(cross_entropy, name='loss')  
        tf.summary.scalar(scope.name+'/loss', loss)  
    return loss  
  
#--------------------------------------------------------------------------  
#loss损失值优化  
    #输入参数：loss。learning_rate，学习速率。  
    #返回参数：train_op，训练op，这个参数要输入sess.run中让模型去训练。  
def trainning(loss, learning_rate):  
    with tf.name_scope('optimizer'):  
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)  
        global_step = tf.Variable(0, name='global_step', trainable=False)  
        train_op = optimizer.minimize(loss, global_step= global_step)  
    return train_op  
  
#-----------------------------------------------------------------------  
#评价/准确率计算  
    #输入参数：logits，网络计算值。labels，标签，也就是真实值，在这里是0或者1。  
    #返回参数：accuracy，当前step的平均准确率，也就是在这些batch中多少张图片被正确分类了。  
def evaluation(logits, labels):  
    with tf.variable_scope('accuracy') as scope:  
        correct = tf.nn.in_top_k(logits, labels, 1)  
        correct = tf.cast(correct, tf.float16)  
        accuracy = tf.reduce_mean(correct)  
        tf.summary.scalar(scope.name+'/accuracy', accuracy)  
    return accuracy  
  
#========================================================================</span>  
def print_confusion_matrix(logits, labels):
    with tf.variable_scope('cm') as scope:  
        cls_true = np.array(labels)
        cls_pred = np.array(logits)
        cm = confusion_matrix(y_true=cls_true,
                              y_pred=cls_pred)  
        tf.summary.scalar(scope.name+'/cm', cm)  
    return cm

In [18]:
#======================================================================  
#导入文件  
import os  
import numpy as np  
import tensorflow as tf  
#import input_data  
#import model  
  
#变量声明  
N_CLASSES = 2  #bhg,hg  
IMG_W = 64   # resize图像，太大的话训练时间久  
IMG_H = 64  
BATCH_SIZE =200  
CAPACITY = 400  
MAX_STEP = 1000 # 一般大于10K  
learning_rate = 0.0001 # 一般小于0.0001  
  
#获取批次batch  
train_dir = 'C:/Users/sungroup/PycharmProjects/anti_cnn/pavement'   #训练样本的读入路径  
logs_train_dir = 'C:/Users/sungroup/PycharmProjects/anti_cnn'    #logs存储路径  
#logs_test_dir =  'E:/Re_train/image_data/test'        #logs存储路径  
  
#train, train_label = input_data.get_files(train_dir)  
train, train_label, val, val_label = get_files(train_dir, 0.1)  
#训练数据及标签  
train_batch,train_label_batch = get_batch(train, train_label, IMG_W, IMG_H, BATCH_SIZE, CAPACITY)  
#测试数据及标签  
val_batch, val_label_batch = get_batch(val, val_label, IMG_W, IMG_H, BATCH_SIZE, CAPACITY)   
  
#训练操作定义  
train_logits = inference(train_batch, BATCH_SIZE, N_CLASSES)  
train_loss = losses(train_logits, train_label_batch)          
train_op = trainning(train_loss, learning_rate)  
train_acc = evaluation(train_logits, train_label_batch)  
  
#测试操作定义  
test_logits = inference(val_batch, BATCH_SIZE, N_CLASSES)  
test_loss = losses(test_logits, val_label_batch)          
test_acc = evaluation(test_logits, val_label_batch)  
test_cm = print_confusion_matrix(test_logits, val_label_batch)
#这个是log汇总记录  
summary_op = tf.summary.merge_all()   
  
#产生一个会话  
sess = tf.Session()    
#产生一个writer来写log文件  
train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)   
#val_writer = tf.summary.FileWriter(logs_test_dir, sess.graph)   
#产生一个saver来存储训练好的模型  
saver = tf.train.Saver()  
#所有节点初始化  
sess.run(tf.global_variables_initializer())    
#队列监控  
coord = tf.train.Coordinator()  
threads = tf.train.start_queue_runners(sess=sess, coord=coord)  
  
#进行batch的训练  
try:  
    #执行MAX_STEP步的训练，一步一个batch  
    for step in np.arange(MAX_STEP):  
        if coord.should_stop():  
            break  
        #启动以下操作节点，有个疑问，为什么train_logits在这里没有开启？  
        _, tra_loss, tra_acc = sess.run([train_op, train_loss, train_acc])  
          
        #每隔50步打印一次当前的loss以及acc，同时记录log，写入writer     
        if step % 50  == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
            summary_str = sess.run(summary_op)  
            train_writer.add_summary(summary_str, step)  
        #每隔100步，保存一次训练好的模型  
        if (step + 1) == MAX_STEP:
            val_loss, val_acc = sess.run([test_loss, test_acc])
            print('test loss = %.2f, test accuracy = %.2f%%' %(val_loss, val_acc*100.0))
            val_cm= sess.run(test_cm)
            print(val_cm)
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')  
            saver.save(sess, checkpoint_path, global_step=step)  
         
except tf.errors.OutOfRangeError:  
    print('Done training -- epoch limit reached')  
  
finally:  
    coord.request_stop()  
      
#========================================================================</span>  

Step 0, train loss = 0.69, train accuracy = 38.50%
Step 50, train loss = 0.67, train accuracy = 60.01%
Step 100, train loss = 0.64, train accuracy = 66.02%
Step 150, train loss = 0.62, train accuracy = 66.02%
Step 200, train loss = 0.60, train accuracy = 71.48%
Step 250, train loss = 0.58, train accuracy = 70.51%
Step 300, train loss = 0.52, train accuracy = 74.51%
Step 350, train loss = 0.53, train accuracy = 75.00%
Step 400, train loss = 0.58, train accuracy = 71.48%
Step 450, train loss = 0.40, train accuracy = 85.50%
Step 500, train loss = 0.41, train accuracy = 82.52%
Step 550, train loss = 0.41, train accuracy = 79.49%
Step 600, train loss = 0.44, train accuracy = 79.98%
Step 650, train loss = 0.34, train accuracy = 85.99%
Step 700, train loss = 0.32, train accuracy = 85.99%
Step 750, train loss = 0.26, train accuracy = 91.99%
Step 800, train loss = 0.28, train accuracy = 86.52%
Step 850, train loss = 0.18, train accuracy = 93.99%
Step 900, train loss = 0.16, train accuracy = 93.

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

Tensor("batch_1:0", shape=(20, 400, 400, 3), dtype=float32)


In [ ]:
def print_confusion_matrix():
    # Get the true classifications for the test-set.
    cls_true = np.array(val_label_batch)
 
    # Get the predicted classifications for the test-set.
    prediction = sess.run(predicted, feed_dict=feed_dict_test)
    y_pred_cls = tf.argmax(prediction, dimension=1)
    cls_pred = sess.run(y_pred_cls, feed_dict=feed_dict_test)

 
    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)
 
    # Print the confusion matrix as text.

    print(cm)
 
    # Plot the confusion matrix as an image.
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
 
    # Make various adjustments to the plot.

    plt.tight_layout()
    plt.colorbar()
    plt.xticks(np.arange(2),('unqualified','qualified'))
    plt.yticks(np.arange(2),('unqualified','qualified'))
    plt.xlabel('Predicted')
    plt.ylabel('True')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    #prediction = sess.run(predicted, feed_dict=feed).squeeze
    #prediction = sess.run(predicted, feed_dict=feed)
    #y_pred_cls = tf.argmax(y_pred, dimension=1)
    #print(prediction.shape)
    print_confusion_matrix()